In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder , OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
from scipy.stats import randint, uniform

df = pd.read_csv("Jayed Prediction.csv", encoding='ISO-8859-1')

X = df.drop('Jayed', axis=1)
y = df['Jayed']


categorical_features = ['Make', 'Model', 'Year', 'Transmission', 'Fuel', 'Condition', 'Paint']
numerical_features = ['Avarege Kilometars', 'Price']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)
    ])

In [ ]:
model = DecisionTreeClassifier(random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

param_distributions = {
    'model__max_depth': randint(3, 20),
    'model__min_samples_split': randint(2, 10),
    'model__min_samples_leaf': randint(1, 10),
    'model__criterion': ['gini', 'entropy']  
}

random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=50, cv=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
random_search.fit(X_train, y_train)

y_pred = random_search.predict(X_test)

# Evaluate the model
print("Best Parameters:")
print(random_search.best_params_)

print("\nAccuracy  --> :")
print(accuracy_score(y_test, y_pred))

In [3]:
unlabeled_df = pd.read_csv("Unlabeld Jayed.csv", encoding='ISO-8859-1')

X_unlabeled = unlabeled_df

y_unlabeled_pred = random_search.predict(X_unlabeled)

unlabeled_df['Predicted_Jayed'] = y_unlabeled_pred

# Save the predictions to a new CSV file
unlabeled_df.to_csv(r"C:\Users\user\OneDrive\Desktop\Car Price Prediction\Jayed Prediction 2.csv" , index=False)